In [1]:
import cv2
import numpy as np

In [2]:
bg = None
accum_weight = 0.5

ROI_top = 100
ROI_bottom = 500
ROI_right = 150
ROI_left = 550

In [4]:
# function to detect background without hand
def cal_accum_avg(frame, accum_weight):
    global bg

    if bg is None:
        bg = frame.copy().astype("float")
        return None

    cv2.accumulateWeighted(frame, bg, accum_weight)

In [5]:
# function to detect the hand and contours and return the thresholded image and max contours
def segment_hand(frame, threshold = 10):
    global bg

    diff = cv2.absdiff(bg.astype("uint8"), frame)

    _, thresholded = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)

    # Grab the external contours for the image
    contours, hierarchy = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) == 0:
        return None
    else:

        hand_segment_max_cont = max(contours, key=cv2.contourArea)

        return (thresholded, hand_segment_max_cont)

In [10]:
# initializing the object for video capture
logi = cv2.VideoCapture(0, cv2.CAP_DSHOW)

num_frames = 0
element = 0 # change element according the image you want from 0 to 9
num_imgs_taken = 0

while True:
    ret, frame = logi.read()
    frame = cv2.resize(frame, (960, 720))
    
    # filpping the frame to prevent inverted image of captured frame...
    frame = cv2.flip(frame, 1)
    frame_copy = frame.copy()

    roi = frame[ROI_top:ROI_bottom, ROI_right:ROI_left]

    gray_frame = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray_frame = cv2.GaussianBlur(gray_frame, (9, 9), 0)

    if num_frames < 60:
        cal_accum_avg(gray_frame, accum_weight)
        if num_frames <= 59:
            cv2.putText(frame_copy, "Please wait until the background is detected.", (80, 570), cv2.FONT_HERSHEY_SIMPLEX, 0.9,
                        (0, 0, 255), 2)

    # Time to configure the hand specifically into the ROI...
    elif num_frames <= 3000:

        hand = segment_hand(gray_frame)

        cv2.putText(frame_copy, "Show the sign with hand for" + str(element), (80, 570), cv2.FONT_HERSHEY_SIMPLEX, 1,
                    (0, 0, 255), 2)

        # Checking if hand is actually detected by counting number of contours detected...
        if hand is not None:
            thresholded, hand_segment = hand

            # Draw contours around hand segment

            cv2.drawContours(frame_copy, [hand_segment + (ROI_right, ROI_top)], -1, (255, 0, 0), 1)

            cv2.putText(frame_copy, str(num_frames) + " Capturing images For" + " " + str(element), (500, 45), cv2.FONT_HERSHEY_SIMPLEX, 1,
                        (0, 0, 255), 2)

            # Also display the threshold image
            cv2.imshow("Threshold Hand Image", thresholded)
            if num_imgs_taken <= 3000:
                # cv2.imwrite(r"D:\\gesture\\train\\"+str(element)+"\\" + str(num_imgs_taken+300) + '.jpg', thresholded)
                string = r"D:\\Sign Language dataset\\Train\\"+str(element) + "\\" + str(num_imgs_taken) + '.jpg'
                cv2.imwrite(string, thresholded)
            else:
                break
            num_imgs_taken += 1
        else:
            cv2.putText(frame_copy, 'No hand detected...', (200, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)


    # Drawing ROI on frame copy to show on the screen for ease
    cv2.rectangle(frame_copy, (ROI_left, ROI_top), (ROI_right, ROI_bottom), (255, 0, 0), 3)

    cv2.putText(frame_copy, "sign language detection project", (10, 20), cv2.FONT_ITALIC, 0.5, (30, 255, 60), 1)

    # increment the number of frames for tracking
    num_frames += 1

    # Display the frame with segmented hand
    cv2.imshow("Sign Detection", frame_copy)

    # Closing windows with Esc key...(any other key with ord can be used too.)
    k = cv2.waitKey(1) & 0xFF

    if k == 27:
        break

# Releasing camera & destroying all the windows...

cv2.destroyAllWindows()
logi.release()